# Demo harvest LAGO

https://docs.google.com/document/d/1F_VVOiU_Mbf6rXRpmgQU09CHdYE4XLAJ6lrdWR4qYU0/edit

In [ ]:
from sickle import Sickle
from bs4 import BeautifulSoup
from io import StringIO 
import requests

## Get IDs from OAI

In [ ]:
sickle = Sickle("http://datahub.egi.eu/oai_pmh")

In [ ]:
records = sickle.ListIdentifiers(**{
    'metadataPrefix': 'oai_dc',
    'set': '986fe2ab97a6b749fac17eb9e9b38c37chb045',
    'ignore_deleted': True,
    #'from': self.fromdate,
})
for rec in records:
    print(rec.identifier)

In [ ]:
records = sickle.ListRecords(**{
    'metadataPrefix': 'oai_dc',
    'set': '986fe2ab97a6b749fac17eb9e9b38c37chb045',
    'ignore_deleted': True,
    #'from': self.fromdate,
})

share_ids = []
for rec in records:
    # print(rec)
    doc = BeautifulSoup(StringIO(rec.raw), 'xml')
    ids = doc.find_all('identifier')
    # print(ids)
    id = [id for id in ids if id.text.startswith('https://datahub.egi.eu/share')][0]
    # print(id.text)
    share_id = id.text.split('/')[-1]
    print(share_id)
    share_ids.append(share_id)

## Get Record file_id from datahub

In [ ]:
ds_ids = []
for share_id in share_ids:
    url = f"https://datahub.egi.eu/api/v3/onezone/shares/{share_id}/public"
    resp = requests.get(url)
    # print(resp.json())
    ds_id = resp.json()['rootFileId']
    print(ds_id)
    ds_ids.append(ds_id)

## Get contents of dataset

In [ ]:
file_ids = []
for ds_id in ds_ids:
    url = f"https://datahub.egi.eu/api/v3/onezone/shares/data/{ds_id}/children?limit=2"
    resp = requests.get(url)
    # print(resp.json())
    for child in resp.json()['children']:
        file_id = child['id']
        print(file_id)
        file_ids.append(file_id)
    

## Iterate over files

In [ ]:
for ds_id in ds_ids:
    url = f"https://datahub.egi.eu/api/v3/onezone/shares/data/{ds_id}/metadata/json"
    resp = requests.get(url)
    # print(resp.json())
    print("title:", resp.json()['title'])
    print("description:", resp.json()['description'])
    print("number of datasets:", len(resp.json().get('dataset', [])))